In [16]:
import csv
import cv2
import numpy as np

def loadImage(line):
    xs = []
    ys = []
    
    correction = 0.2
    
    for view in range(3):
        origPath = line[view]
        filename = origPath.split('/')[-1]
        newPath = "data/IMG/" + filename
        
        x = cv2.imread(newPath)
        steer = float(line[3])
                       
        if view == 1: # Left view
            steer += correction
        if view == 2: # Right view
            steer -= correction
        
        xs.append(np.array(x))
        ys.append(steer)
        
    
    return xs,ys

# load training file list
lines = []
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
# load images and steering angle
images = []
measurements = []
for line in lines:
    xs, ys = loadImage(line)
    for i in range(len(xs)):
        x = xs[i]
        y = ys[i]
        images.append(x)
        measurements.append(y)
        images.append(cv2.flip(x, 1))
        measurements.append(-1*y)
    
X_train = np.array(images)
y_train = np.array(measurements)

img_shape = X_train[0].shape

print('X size:', X_train.shape)
print('y size:', y_train.shape)
print('img sample size:', img_shape)

X size: (24810, 160, 320, 3)
y size: (24810,)
img sample size: (160, 320, 3)


In [17]:
from keras.models import Sequential
from keras.layers import Input, Flatten, Dense, Lambda, Convolution2D,MaxPooling2D, Dropout, Cropping2D

top_crop = 50
bottom_crop = 20

def AlexNet():
    model = Sequential()
    model.add(Cropping2D(cropping=((top_crop,bottom_crop), (0,0)), input_shape=(160,320,3), dim_ordering='tf'))
    model.add(Lambda(lambda x:x/255.0-0.5))
    model.add(Convolution2D(6,5,5,activation="relu"))
    model.add(MaxPooling2D())
    model.add(Convolution2D(6,5,5,activation="relu"))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(120))
    model.add(Dropout(0.2))
    model.add(Dense(84))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    return model

def NvidiaNet():
    model = Sequential()
    model.add(Cropping2D(cropping=((top_crop,bottom_crop), (0,0)), input_shape=(160,320,3), dim_ordering='tf'))
    model.add(Lambda(lambda x:x/255.0-0.5))
    model.add(Convolution2D(24,5,5, subsample=(2,2),activation="relu"))
    model.add(Convolution2D(36,5,5, subsample=(2,2),activation="relu"))
    model.add(Convolution2D(48,5,5, subsample=(2,2),activation="relu"))
    model.add(Convolution2D(64,3,3, activation="relu"))
    model.add(Convolution2D(64,3,3, activation="relu"))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))
    return model

model = NvidiaNet()
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=10, batch_size=256)

model.save('model.h5')

Train on 19848 samples, validate on 4962 samples
Epoch 1/10
19848/19848 [==============================] - 79s - loss: 0.0330 - val_loss: 0.0368
Epoch 2/10
19848/19848 [==============================] - 79s - loss: 0.0225 - val_loss: 0.0343
Epoch 3/10
19848/19848 [==============================] - 79s - loss: 0.0190 - val_loss: 0.0353
Epoch 4/10
19848/19848 [==============================] - 79s - loss: 0.0160 - val_loss: 0.0397
Epoch 5/10
19848/19848 [==============================] - 79s - loss: 0.0133 - val_loss: 0.0449
Epoch 6/10
19848/19848 [==============================] - 79s - loss: 0.0115 - val_loss: 0.0468
Epoch 7/10
19848/19848 [==============================] - 79s - loss: 0.0102 - val_loss: 0.0489
Epoch 8/10
19848/19848 [==============================] - 79s - loss: 0.0087 - val_loss: 0.0477
Epoch 9/10
19848/19848 [==============================] - 78s - loss: 0.0077 - val_loss: 0.0464
Epoch 10/10
19848/19848 [==============================] - 78s - loss: 0.0069 - val_los